In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Load datasets
train_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
test_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)
abnormal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_abnormal.csv', header=None)
normal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_normal.csv', header=None)

# Combine the abnormal and normal data
normal_data['label'] = 0
abnormal_data['label'] = 1

# Combine all datasets
combined_data = pd.concat([train_data, test_data, normal_data, abnormal_data], axis=0)

# Split features and labels
X = combined_data.iloc[:, :-1].values
y = combined_data.iloc[:, -1].values

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape for CNN (if data is time-series, reshape accordingly)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Check class distribution
print(combined_data['label'].value_counts())

In [ ]:
import numpy as np
import pandas as pd

# Load datasets
train_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)
test_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
abnormal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_abnormal.csv', header=None)
normal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_normal.csv', header=None)

# Combine the abnormal and normal data
normal_data['label'] = 0
abnormal_data['label'] = 1

# Combine all datasets
combined_data = pd.concat([train_data, test_data, normal_data, abnormal_data], axis=0)

# Check and handle missing values
print("Missing values in labels:", combined_data['label'].isna().sum())
print("Missing values in features:", combined_data.isna().sum().sum())

# Drop rows with NaN values in the label column
combined_data = combined_data.dropna(subset=['label'])

# Check again
print("Missing values in cleaned labels:", combined_data['label'].isna().sum())
print("Missing values in cleaned features:", combined_data.isna().sum().sum())

# Split features and labels
X = combined_data.iloc[:, :-1].values
y = combined_data.iloc[:, -1].values

# Confirm no NaNs in y
print("NaNs in y:", np.isnan(y).sum())

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Load datasets
train_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)
test_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
abnormal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_abnormal.csv', header=None)
normal_data = pd.read_csv('/kaggle/input/heartbeat/ptbdb_normal.csv', header=None)


# Combine the abnormal and normal data
normal_data['label'] = 0
abnormal_data['label'] = 1

# Combine all datasets
combined_data = pd.concat([train_data, test_data, normal_data, abnormal_data], axis=0)

# Check and handle missing values
print("Missing values in labels:", combined_data['label'].isna().sum())
print("Missing values in features:", combined_data.isna().sum().sum())

# Drop rows with NaN values in the label column
combined_data = combined_data.dropna(subset=['label'])

# Verify again after cleaning
print("Missing values in cleaned labels:", combined_data['label'].isna().sum())
print("Missing values in cleaned features:", combined_data.isna().sum().sum())

# Split features and labels
X = combined_data.iloc[:, :-1].values
y = combined_data.iloc[:, -1].values

# Confirm no NaNs in y
print("NaNs in y:", np.isnan(y).sum())

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Resample data to handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Reshape for CNN
X_resampled = X_resampled.reshape((X_resampled.shape[0], X_resampled.shape[1], 1))

# Split resampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_resampled), y=y_resampled)
class_weight_dict = dict(enumerate(class_weights))

#Building the model
model = Sequential([
    Conv1D(120, 5, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Dropout(0.45),
    Conv1D(240, 5, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.45),
    Flatten(),
    Dense(240, activation='relu'),
    Dropout(0.45),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model with class weights
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, class_weight=class_weight_dict)

# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# Predict and evaluate
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Extract training and validation accuracy and loss from the history object
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Print training and validation accuracy and loss
print("\nTraining and Validation Accuracy:")
for epoch in range(len(train_acc)):
    print(f"Epoch {epoch + 1}: Training Accuracy = {train_acc[epoch]:.4f}, Validation Accuracy = {val_acc[epoch]:.4f}")

print("\nTraining and Validation Loss:")
for epoch in range(len(train_loss)):
    print(f"Epoch {epoch + 1}: Training Loss = {train_loss[epoch]:.4f}, Validation Loss = {val_loss[epoch]:.4f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Predict and evaluate
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Abnormal'], yticklabels=['Normal', 'Abnormal'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Extract training and validation accuracy and loss from the history object
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot training and validation accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
data_test = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)

In [ ]:
# Separate features and labels
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
X_test = data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, -1].values

In [ ]:
# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [ ]:
# Convert signals to 2D spectrogram images
def signal_to_spectrogram(signal):
    f, t, Sxx = spectrogram(signal, fs=100, nperseg=64, noverlap=32)
    return Sxx

In [ ]:
# Convert the entire dataset to spectrograms
X_spectrograms = np.array([signal_to_spectrogram(signal) for signal in X])
X_test_spectrograms = np.array([signal_to_spectrogram(signal) for signal in X_test])

In [ ]:
# Reshape for Conv2D input
X_spectrograms = X_spectrograms[..., np.newaxis]  # Add channel dimension
X_test_spectrograms = X_test_spectrograms[..., np.newaxis]

In [ ]:
# Convert labels to categorical
y_categorical = to_categorical(y)
y_test_categorical = to_categorical(y_test)

In [ ]:
# Split the training data
X_train, X_val, y_train, y_val = train_test_split(X_spectrograms, y_categorical, test_size=0.2, random_state=42)

In [ ]:
# Build 2D CNN model with 'same' padding
model_2d_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_categorical.shape[1], activation='softmax')
])

In [ ]:
model_2d_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model_2d_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model_2d_cnn.evaluate(X_test_spectrograms, y_test_categorical)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

## Stopping early using TensorFlow EarlyStopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy',  # Monitor validation accuracy
                               patience=5,  # Stop training if no improvement after 5 epochs
                               restore_best_weights=True)  # Restore the model weights from the epoch with the best validation accuracy

# Train the model with early stopping
history = model_2d_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # Set a higher number of epochs, as early stopping will stop training once it doesn't improve
    batch_size=32,
    callbacks=[early_stopping]  # Add the early stopping callback
)

# Evaluate the model on test data
test_loss, test_accuracy = model_2d_cnn.evaluate(X_test_spectrograms, y_test_categorical)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()